In [1]:
import os
import pymongo
import boto3
import json
import pandas as pd
import numpy as np
from textblob import TextBlob
from utils.DocDB import get_conversation_history
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
def create_msg_dataset(df):
    user_messages = df[df['role'] == 'user'][['conv_id', 'msg_id', 'time', 'query', 'rewritten_query']]
    assistant_responses = df[df['role'] == 'assistant'][['conv_id', 'msg_id', 'response', 'metadata', 'reference']]
    assistant_responses['msg_id'] = assistant_responses['msg_id'] - 1
    custom_df = pd.merge(user_messages, assistant_responses, on=['conv_id', 'msg_id'], how='left', suffixes=('_user', '_assistant'))
    custom_df = custom_df.sort_values(by=['conv_id', 'msg_id'], ascending=[True, True])
    return custom_df

In [3]:
def get_unique_users(df):
    unique_users = df['conv_id'].nunique()
    print(f"Total unique users: {unique_users}")
    most_frequent_users = df['conv_id'].value_counts().head(5)
    # print(most_frequent_users)
    return unique_users, most_frequent_users


def get_unique_queries(df):
    unique_questions = df['query'].nunique()
    print(f"Total unique questions: {unique_questions}")
    most_frequent_questions = df['query'].value_counts().head(5)
    # print(most_frequent_questions)
    return unique_questions, most_frequent_questions


def analyze_conversation_flow(df):
    conv_turns = df.groupby('conv_id').size()   # Calculate message count per conversation
    avg_turns = round(conv_turns.mean())
    # print(f"Average turns per conversation: {avg_turns}")
    return conv_turns, avg_turns


def analyse_user_sentiment(df):
    df['sentiment'] = df['query'].apply(lambda x: TextBlob(x).sentiment.polarity)
    # print(df[['query', 'sentiment']].head(5))
    return df['sentiment']


def calculate_failure_rate(df):
    failed_queries = df[df['response'].isnull()].shape[0]
    total_queries = df.shape[0]
    failure_rate = failed_queries / total_queries
    return failure_rate


def get_response_accuracy(df):
    def calculate_accuracy(row):
        if pd.isna(row['response']) and not pd.isna(row['ground_truth_response']):
            return 0
        elif isinstance(row['response'], str) and row['response'] in row['ground_truth_response']:
            return 1
        elif isinstance(row['response'], str) and row['response'] and row['ground_truth_response'] and \
             any(word in row['ground_truth_response'] for word in row['response'].split()):
            return 0.5
        else:
            return 0
    df['response_accuracy'] = df.apply(calculate_accuracy, axis=1)
    accuracy = df['response_accuracy'].mean()
    return df['response_accuracy'], accuracy


def get_rewrite_accuracy(df):
    # Assuming you have a column 'correct_intent' with labeled intents
    df['intent_accuracy'] = df.apply(lambda x: 1 if x['query'] in x['rewritten_query'] else 0, axis=1)
    intent_accuracy = df['intent_accuracy'].mean()
    print(f"Intent recognition accuracy: {intent_accuracy}")
    return intent_accuracy


def analyze_session(df):
    df['time'] = pd.to_datetime(df['time'])
    session_length = df.groupby('conv_id')['time'].apply(lambda x: x.max() - x.min())
    # print(f"Session length (time difference between first and last message):{session_length}")
    return session_length


def extract_top_keywords(df):
    vectorizer = CountVectorizer(max_features=10, stop_words='english')
    X = vectorizer.fit_transform(df['query'])
    keywords = vectorizer.get_feature_names_out()
    # print(f"Top keywords: {keywords}")
    return keywords

In [4]:
from bedrock_call import get_prompt
from docdbVS import similarity_search
from app import create_embeddings, call_llm

def clean_raw_response(raw_response):
    try:
        cleaned_response = raw_response.strip()
        if cleaned_response.startswith('"""') and cleaned_response.endswith('"""'):
            cleaned_response = cleaned_response.strip('"""')
            response = json.loads(cleaned_response)
        return response
    except Exception as e:
        prompt = f"You need to correct the JSON format of this raw_response to be parsed successfully. raw_response:{raw_response}"
        response = call_llm(client, prompt)
        return response

def create_ground_truth(db, unique_questions):
    ground_truth_responses = []
    ground_truth_metadata = []
    ground_truth_references = []

    for rewritten_query in unique_questions:
        try:
            client = boto3.client(service_name='bedrock-runtime', region_name="ap-south-1")
            embed_client = boto3.client(service_name='bedrock-runtime', region_name="us-west-2")
            print("Creating embeddings for the query.")
            query_embedding = create_embeddings(embed_client, rewritten_query)

            if query_embedding is None:
                print("Error: Failed to create embeddings for the query.")
                ground_truth_responses.append(None)
                ground_truth_metadata.append(None)
                ground_truth_references.append(None)
                continue

            similar_docs = similarity_search(db['Test'], embedding=query_embedding, embedding_key='embedding', text_key='text', k=5)
            prompt = get_prompt(rewritten_query, similar_docs, None)
            response = call_llm(client, prompt, model='meta.llama3-70b-instruct-v1:0')

            if response is None:
                print("Error: Failed to generate a response from LLM.")
                ground_truth_responses.append(None)
                ground_truth_metadata.append(None)
                ground_truth_references.append(None)
                continue  # Skip this iteration

            # Parse the response and validate the structure
            try:
                parsed_response = json.loads(response)
                print("LLM response parsed successfully.")
            except json.JSONDecodeError:
                print("Error: JSONDecodeError occurred. Attempting to clean response.")
                result = clean_raw_response(response)
                try:
                    parsed_response = json.loads(result)
                except json.JSONDecodeError as e:
                    print(f"Error: Failed to parse LLM response: {e}")
                    ground_truth_responses.append(None)
                    ground_truth_metadata.append(None)
                    ground_truth_references.append(None)
                    continue  # Skip this iteration

            # Append parsed response data
            ground_truth_responses.append(parsed_response.get('Answer'))
            ground_truth_metadata.append(parsed_response.get('Metadata'))
            ground_truth_references.append(parsed_response.get('References'))

        except Exception as e:
            print(f"Error: Unexpected error in create_ground_truth: {e}")
            ground_truth_responses.append(None)
            ground_truth_metadata.append(None)
            ground_truth_references.append(None)

    # Create a DataFrame to hold the ground truth responses for unique questions
    results_df = pd.DataFrame({
        'rewritten_query': unique_questions,
        'ground_truth_response': ground_truth_responses,
        'ground_truth_metadata': ground_truth_metadata,
        'ground_truth_reference': ground_truth_references
    })

    return results_df

In [5]:
# Fetch and analyze data
df = get_conversation_history()

INFO:pymongo.client:You appear to be connected to a DocumentDB cluster. For more information regarding feature compatibility and support please visit https://www.mongodb.com/supportability/documentdb


In [6]:
def get_eval_prompt(row):
    prompt = f"""
Evaluate the following response based on these metrics: faithfulness, context precision, answer correctness, answer relevancy, and context recall.

**Question:** {row['rewritten_query']}
**Response:** {row['response']}
**Ground Truth:** {row['ground_truth_response']}
**Contexts:** {row['ground_truth_reference'] if row['ground_truth_reference'] else "No context available."}

Please provide scores for each metric in JSON format as shown below:
{{
    "Faithfulness": "<Score between 0 and 1>",
    "Context_precision": "<Score between 0 and 1>",
    "Answer_correctness": "<Score between 0 and 1>",
    "Answer_relevancy": "<Score between 0 and 1>",
    "Context_recall": "<Score between 0 and 1>",
    "Reason": "<Brief explanation (max 20 words)>"
}}

Return only the output JSON without any additional text.
"""
    return prompt

In [7]:
def ragas_analysis(df):
    df['response'].fillna("", inplace=True)  # Avoid chaining warnings
    scores = { 
        'faithfulness': [], 
        'context_precision': [], 
        'answer_correctness': [], 
        'answer_relevancy': [], 
        'context_recall': [], 
        'reason': []
    }
    for index, row in df.iterrows():
        client = boto3.client(service_name='bedrock-runtime', region_name="ap-south-1")
        eval_prompt = get_eval_prompt(row)
        scores_response = call_llm(client, eval_prompt, model='meta.llama3-70b-instruct-v1:0')
        print(scores_response)
        # Handle possible JSON response
        try:
            evaluation_scores = json.loads(scores_response)
            print(evaluation_scores)
            for key in scores.keys():
                print(key)
                scores[key].append(evaluation_scores.get(key.capitalize(), None))
                print(evaluation_scores.get(key.capitalize()))
        except json.JSONDecodeError:
            print(f"Error: Failed to parse evaluation scores for index {index}")
    for key, value in scores.items():
        df[key] = value

    return df

In [8]:
def analyze_history(db, df=None, file_name=None, save_file="analysis_report.xlsx"):
    if file_name:
        df = pd.read_excel(file_name)

    # Perform analysis
    unique_users, most_frequent_users = get_unique_users(df)
    unique_questions, most_frequent_questions = get_unique_queries(df)
    conv_size, avg_conv_size = analyze_conversation_flow(df)
    df['sentiment'] = analyse_user_sentiment(df)
    session_length = analyze_session(df)
    most_inquired_topics = extract_top_keywords(df)

    # Most asked unique questions
    top_questions = df['rewritten_query'].value_counts().head(20).index.tolist()    
    ground_truth_df = create_ground_truth(db, top_questions)

    # Filter and merge
    custom_df = df[df['rewritten_query'].isin(top_questions)].copy()
    custom_df = custom_df.merge(ground_truth_df, on='rewritten_query', how='left')
    custom_df = custom_df.drop_duplicates(subset='rewritten_query')

    # Calculate additional metrics
    failure_rate = calculate_failure_rate(df)
    rewrite_accuracy = get_rewrite_accuracy(df)
    custom_df['response_accuracy'], avg_accuracy = get_response_accuracy(custom_df)

    print(custom_df['response_accuracy'])
    custom_df = ragas_analysis(custom_df)
    print(custom_df)
    
    # Check if the file already exists and Save analysis results to an Excel file
    if not os.path.exists(save_file):
        with pd.ExcelWriter(save_file, engine='openpyxl', mode='w') as writer:
            df.to_excel(writer, sheet_name='Data w sentiment analysis', index=False)
    else:
        with pd.ExcelWriter(save_file, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
            df.to_excel(writer, sheet_name='Data w sentiment analysis', index=False)
    with pd.ExcelWriter(save_file, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
        custom_df.to_excel(writer, sheet_name='Data w Ground Truth', index=False)
        # Save user analysis
        user_data = most_frequent_users.reset_index().rename(columns={'conv_id': 'User', 'count': 'Count'})
        user_data.to_excel(writer, sheet_name='User Analysis', index=False)
        
        # Save query analysis
        query_data = most_frequent_questions.reset_index().rename(columns={'query': 'Query', 'count': 'Count'})
        query_data.to_excel(writer, sheet_name='Query Analysis', index=False)
        
        # Save conversation flow analysis
        combined_data = pd.DataFrame({
            'conv_id': conv_size.index,
            'Interaction Count': conv_size.values,
            'Session Length': session_length.values
        })
        combined_data.to_excel(writer, sheet_name='Conversation Analysis', index=False)

        # Save keywords
        keywords_data = pd.DataFrame(most_inquired_topics, columns=['Keyword'])
        keywords_data.to_excel(writer, sheet_name='Top Keywords', index=False)

        # Save failure rate and accuracy
        summary = pd.DataFrame({
            'Metric': ['Failure Rate', 'Rewrite Accuracy', 'Response Accuracy'],
            'Value': [failure_rate, rewrite_accuracy, avg_accuracy]
        })
        summary.to_excel(writer, sheet_name='Summary', index=False)

    print(f"Analysis successfully saved to {save_file}")

In [9]:
if df is not None:
    df.to_excel('conversation_data.xlsx', index=False)
    print(f"Data successfully saved to conversation_data.xlsx")
    conv = create_msg_dataset(df)
    conv.to_excel('conv_history.xlsx', index=False)
    print(f"Data successfully saved to conv_history.xlsx")
analyze_history(db, file_name = 'conv_history.xlsx')

Data successfully saved to conversation_data.xlsx
Data successfully saved to conv_history.xlsx
Total unique users: 24
Total unique questions: 197


INFO:botocore.credentials:Found credentials from IAM Role: ACCESS_TO_AWS_SERVICES


Creating embeddings for the query.


INFO:app:LLM request received
INFO:app:LLM Call 0
INFO:app:Raw Response: {'ResponseMetadata': {'RequestId': '982d7b3e-d5ab-4e55-8b03-b52424bfe275', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Thu, 26 Sep 2024 11:12:42 GMT', 'content-type': 'application/json', 'content-length': '646', 'connection': 'keep-alive', 'x-amzn-requestid': '982d7b3e-d5ab-4e55-8b03-b52424bfe275', 'x-amzn-bedrock-invocation-latency': '1861', 'x-amzn-bedrock-output-token-count': '111', 'x-amzn-bedrock-input-token-count': '1205'}, 'RetryAttempts': 0}, 'contentType': 'application/json', 'body': <botocore.response.StreamingBody object at 0xf602cab45de0>}
INFO:app:Response Data: {"generation":"\n\n{\n    \"Answer\": \"The number of leaves in Gemini Solutions is not explicitly mentioned in the provided texts. However, according to the company's policies, employees are entitled to a certain number of leaves. Unfortunately, the exact number is not specified in the given texts.\",\n    \"References\": [],\n    \"Metad

LLM response parsed successfully.
Creating embeddings for the query.


...



{
    "Faithfulness": "0",
    "Context_precision": "0",
    "Answer_correctness": "0",
    "Answer_relevancy": "0",
    "Context_recall": "0",
    "Reason": "Response does not match the expected greeting and introduction."
}
{'Faithfulness': '0', 'Context_precision': '0', 'Answer_correctness': '0', 'Answer_relevancy': '0', 'Context_recall': '0', 'Reason': 'Response does not match the expected greeting and introduction.'}
faithfulness
0
context_precision
0
answer_correctness
0
answer_relevancy
0
context_recall
0
reason
Response does not match the expected greeting and introduction.


INFO:app:Raw Response: {'ResponseMetadata': {'RequestId': 'ef662645-3447-4226-a7db-15936211dc88', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Thu, 26 Sep 2024 11:14:38 GMT', 'content-type': 'application/json', 'content-length': '348', 'connection': 'keep-alive', 'x-amzn-requestid': 'ef662645-3447-4226-a7db-15936211dc88', 'x-amzn-bedrock-invocation-latency': '993', 'x-amzn-bedrock-output-token-count': '61', 'x-amzn-bedrock-input-token-count': '369'}, 'RetryAttempts': 0}, 'contentType': 'application/json', 'body': <botocore.response.StreamingBody object at 0xf602cab2dd80>}
INFO:app:Response Data: {"generation":"\n\n{\n    \"Faithfulness\": \"1\",\n    \"Context_precision\": \"1\",\n    \"Answer_correctness\": \"1\",\n    \"Answer_relevancy\": \"1\",\n    \"Context_recall\": \"1\",\n    \"Reason\": \"Response accurately summarizes the travel policy guidelines.\"\n}","prompt_token_count":369,"generation_token_count":61,"stop_reason":"stop"}
INFO:app:LLM request received
INFO:app:LLM Ca



{
    "Faithfulness": "1",
    "Context_precision": "1",
    "Answer_correctness": "1",
    "Answer_relevancy": "1",
    "Context_recall": "1",
    "Reason": "Response accurately summarizes the travel policy guidelines."
}
{'Faithfulness': '1', 'Context_precision': '1', 'Answer_correctness': '1', 'Answer_relevancy': '1', 'Context_recall': '1', 'Reason': 'Response accurately summarizes the travel policy guidelines.'}
faithfulness
1
context_precision
1
answer_correctness
1
answer_relevancy
1
context_recall
1
reason
Response accurately summarizes the travel policy guidelines.


INFO:app:Raw Response: {'ResponseMetadata': {'RequestId': 'b386288e-8cc0-48f7-b30f-a047e74ddb1d', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Thu, 26 Sep 2024 11:14:39 GMT', 'content-type': 'application/json', 'content-length': '346', 'connection': 'keep-alive', 'x-amzn-requestid': 'b386288e-8cc0-48f7-b30f-a047e74ddb1d', 'x-amzn-bedrock-invocation-latency': '1112', 'x-amzn-bedrock-output-token-count': '68', 'x-amzn-bedrock-input-token-count': '388'}, 'RetryAttempts': 0}, 'contentType': 'application/json', 'body': <botocore.response.StreamingBody object at 0xf602cab2eec0>}
INFO:app:Response Data: {"generation":"\n\n{\n    \"Faithfulness\": 0.8,\n    \"Context_precision\": 1,\n    \"Answer_correctness\": 0,\n    \"Answer_relevancy\": 1,\n    \"Context_recall\": 1,\n    \"Reason\": \"Incorrect calculation of total leaves, but relevant and faithful to context.\"\n}","prompt_token_count":388,"generation_token_count":68,"stop_reason":"stop"}




{
    "Faithfulness": 0.8,
    "Context_precision": 1,
    "Answer_correctness": 0,
    "Answer_relevancy": 1,
    "Context_recall": 1,
    "Reason": "Incorrect calculation of total leaves, but relevant and faithful to context."
}
{'Faithfulness': 0.8, 'Context_precision': 1, 'Answer_correctness': 0, 'Answer_relevancy': 1, 'Context_recall': 1, 'Reason': 'Incorrect calculation of total leaves, but relevant and faithful to context.'}
faithfulness
0.8
context_precision
1
answer_correctness
0
answer_relevancy
1
context_recall
1
reason
Incorrect calculation of total leaves, but relevant and faithful to context.
                                               conv_id  msg_id  \
0                                                  123       1   
1                                               123456       1   
2                                               123456       3   
3                                               123456       7   
4                                               123456